# **Hypothesis Testing with Men's and Women's Soccer Matches**

You're working as a sports journalist at a major online sports media company, specializing in soccer analysis and reporting. You've been watching both men's and women's international soccer matches for a number of years, and your gut instinct tells you that more goals are scored in women's international football matches than men's. This would make an interesting investigative article that your subscribers are bound to love, but you'll need to perform a valid statistical hypothesis test to be sure!

While scoping this project, you acknowledge that the sport has changed a lot over the years, and performances likely vary a lot depending on the tournament, so you decide to limit the data used in the analysis to only official FIFA World Cup matches (not including qualifiers) since 2002-01-01.

You create two datasets containing the results of every official men's and women's international football match since the 19th century, which you scraped from a reliable online source. This data is stored in two CSV files: women_results.csv and men_results.csv.

The question you are trying to determine the answer to is:

Are more goals scored in women's international soccer matches than men's?

You assume a 10% significance level, and use the following null and alternative hypotheses:

 H0: The mean number of goals scored in women's international soccer matches is the same as men's.

 Ha: The mean number of goals scored in women's international soccer matches is greater than men's.

#### Project Instructions:

Perform an appropriate hypothesis test to determine the p-value, and hence result, of whether to reject or fail to reject the null hypothesis that the mean number of goals scored in women's international soccer matches is the same as men's. Use a 10% significance level.

For this analysis, you'll use Official FIFA World Cup matches since 2002-01-01, and you'll also assume that each match is fully independent, i.e., team form is ignored.

The p-value and the result of the test must be stored in a dictionary called result_dict in the form:

result_dict = {"p_val": p_val, "result": result}

where p_val is the p-value and result is either the string "fail to reject" or "reject", depending on the result of the test.

In [2]:
# Loading in required libraries

import os
import pandas as pd
import seaborn as sns
import numpy as np

# Reading in the Nobel Prize data
# build read_csv function
folder_name = 'datasets'
dir = r'C:\Users\mcaba\OneDrive\Escritorio\Data Science\Datacamp_Projects\DataCamp_Projects\{}'.format(folder_name)

def read_csv_fun(folder_name,file_name, path):
    path = dir
    os.chdir(path)
    df = pd.read_csv('{}.csv'.format(file_name), sep=',', low_memory=False, on_bad_lines='skip')
    return df

women_results = read_csv_fun('datasets','women_results', dir)

# Taking a look at the first several winners
women_results.head(2)

,Unnamed: 0,date,home_team,away_team,home_score,away_score,tournament
0,0,1969-11-01,Italy,France,1,0,Euro
1,1,1969-11-01,Denmark,England,4,3,Euro


In [3]:
men_results = read_csv_fun('datasets','men_results', dir)

# Taking a look at the first several winners
men_results.head(2)

,Unnamed: 0,date,home_team,away_team,home_score,away_score,tournament
0,0,1872-11-30,Scotland,England,0,0,Friendly
1,1,1873-03-08,England,Scotland,4,2,Friendly


In [8]:
display(women_results.info())

display(women_results.describe())

display(men_results.info())

display(men_results.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4884 entries, 0 to 4883
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  4884 non-null   int64         
 1   date        4884 non-null   datetime64[ns]
 2   home_team   4884 non-null   object        
 3   away_team   4884 non-null   object        
 4   home_score  4884 non-null   int64         
 5   away_score  4884 non-null   int64         
 6   tournament  4884 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 267.2+ KB


None

,Unnamed: 0,date,home_score,away_score
count,4884.000000,4884,4884.000000,4884.000000
mean,2441.500000,2007-08-17 22:29:11.351351296,2.272727,1.431409
min,0.000000,1969-11-01 00:00:00,0.000000,0.000000
25%,1220.750000,2000-06-23 00:00:00,0.000000,0.000000
50%,2441.500000,2009-07-13 00:00:00,1.000000,1.000000
75%,3662.250000,2016-04-12 00:00:00,3.000000,2.000000
max,4883.000000,2022-07-31 00:00:00,24.000000,24.000000
std,1410.033688,NaN,2.736377,1.974651


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44353 entries, 0 to 44352
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  44353 non-null  int64         
 1   date        44353 non-null  datetime64[ns]
 2   home_team   44353 non-null  object        
 3   away_team   44353 non-null  object        
 4   home_score  44353 non-null  int64         
 5   away_score  44353 non-null  int64         
 6   tournament  44353 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 2.4+ MB


None

,Unnamed: 0,date,home_score,away_score
count,44353.000000,44353,44353.000000,44353.000000
mean,22176.000000,1992-06-23 17:28:36.763240320,1.740559,1.178793
min,0.000000,1872-11-30 00:00:00,0.000000,0.000000
25%,11088.000000,1979-05-13 00:00:00,1.000000,0.000000
50%,22176.000000,1999-03-11 00:00:00,1.000000,1.000000
75%,33264.000000,2011-03-25 00:00:00,2.000000,2.000000
max,44352.000000,2022-12-18 00:00:00,31.000000,21.000000
std,12803.752581,NaN,1.748722,1.394580


In [7]:
#Transforming data columns into the correct Dtype

women_results["date"] = pd.to_datetime(women_results["date"])

men_results["date"] = pd.to_datetime(men_results["date"])

In [ ]:
# year column for validation

women_results["year"] = 

men_results["year"] = 

In [ ]:
# Subset the DataFrame for type "Movie"
netflix_df_movies_only = netflix_df[netflix_df.type == 'Movie']